In [ ]:
# Question 1: List all visits with at least one click.

#Option 1:

SELECT distinct visitId
  FROM clicks

# Option 2:

SELECT distinct s.visitId
  FROM searches s 
  LEFT JOIN clicks c on c.searchId = s.id
  WHERE clickId is not NULL

# Option 3:

SELECT s.visitId
  FROM searches s 
  LEFT JOIN clicks c on c.searchId = s.id
  GROUP BY 1
  HAVING count(clickId) > 0




In [ ]:
# Question 2: The percentage of search having clicks per day, over the last 7 days, including overall summary value, using single SQL query.
## Assuming it would be the same timezone for all countries (our timezone) and today is the last day of the datetime dimension in the search table.

# If we use Postgres, then:

select count(clickId)::numeric / count(id) as overall_click_rate,
       count( CASE WHEN
              current_date() - datetime::date <= 7 then clickId
              ELSE NULL END)::numeric / count(CASE WHEN
              current_date() - datetime::date <= 7 then id
              ELSE NULL END) as click_rate_7_days

from search s
left join on clicks c on c.searchId = s.id

# if we use MySQL, then:

select CAST(count(clickId) AS FLOAT) / count(id) as overall_click_rate,
       CAST(count( CASE WHEN
              (DATEDIFF(NOW(), cast(datetime as date)) <= 7 then clickId
              ELSE NULL END) AS FLOAT) / count(CASE WHEN
              DATEDIFF(NOW(), cast(datetime as date)) <= 7 then id
              ELSE NULL END) as click_rate_7_days

from search s
left join on clicks c on c.searchId = s.id


# Let's cast the datetime so that we take different timezones into consideration. Assuming that the data is in UTC and we want to analyze China only with its respective time zone (CCT):

# If we use Postgres, then:

select 
count(clickId)::numeric / count(id) as overall_click_rate,
       count( CASE WHEN
              current_date() AT TIME ZONE 'UTC' AT TIME ZONE 'CCT' - (datetime AT TIME ZONE 'UTC' AT TIME ZONE 'CCT')::date <= 7 then clickId
              ELSE NULL END)::numeric / count(CASE WHEN
              current_date() AT TIME ZONE 'UTC' AT TIME ZONE 'CCT' - (datetime AT TIME ZONE 'UTC' AT TIME ZONE 'CCT')::date <= 7 then id
              ELSE NULL END) as click_rate_7_days

from search s
left join on clicks c on c.searchId = s.id

# If we use MySQL, then:

select CAST(count(clickId) AS FLOAT) / count(id) as overall_click_rate,
       CAST(count( CASE WHEN
              (DATEDIFF(CONVERT_TZ(NOW(), '+00:00', '+08:00'), cast(datetime as date)) <= 7 then clickId
              ELSE NULL END) AS FLOAT) / count(CASE WHEN
              DATEDIFF(CONVERT_TZ(NOW(), '+00:00', '+08:00'), cast(datetime as date)) <= 7 then id
              ELSE NULL END) as click_rate_7_days

from search s
left join on clicks c on c.searchId = s.id


In [ ]:
# Question 3: List the 10 most active users (by number of searches) having at least 1 click on a document coming from a source starting with “Confluence”.

SELECT s.username,      
  FROM searches s
  LEFT JOIN clicks c on c.searchId = s.id
  WHERE c.sourceName LIKE '%Confluence%'
  GROUP BY 1
  HAVING count(clickId) >= 1
  ORDER BY count(id) DESC
  LIMIT 10

In [ ]:
# Question 4: List the top 10 most popular items and display their average click rank.

## Here I will use the number of visits instead of number of clicks since an item could be opened more than one time per visit. - I will prove it using the second query.

SELECT documentTitle,
       count(distinct visitId) as num_visits,
       AVG(clickRank) as avg_click_rank
       FROM clicks
       GROUP BY 1
       ORDER BY 2 DESC
       LIMIT 10


# --> Documents (items) with more than 1 click per visit

WITH base_table AS (
    SELECT documentTitle,
    count(distinct visitId) as num_visits,
    count(clickId) as num_clicks
    FROM clicks
    GROUP BY 1
    ORDER BY 2 DESC
    )

SELECT *
FROM base_table
WHERE (num_clicks - num_visits) != 0


# Extra SQL Queries

In [ ]:
# Knowing that a click-through rate of below 40% is bad and should be looked into, between 40% and 60% is a gray zone and should be under watch and about 60% is optimal, let's give each query 
#one of the following category: Bad if < 40%, okish if 40% < > 60% and splendid if > 60%. Moreover, let's sort the list by number of searches.

WITH click_rate_table as (
   SELECT queryExpression,
   count(id) as num_searches,
   round((count(clickId)*1.0 / (count(id))) * 100, 2) as click_rate
   FROM searches s
   LEFT JOIN clicks c on c.searchId = s.id
   GROUP BY 1
   ORDER BY 2 DESC )
                        
   SELECT *,
          CASE 
           WHEN click_rate < 40 then 'Bad'
           WHEN click_rate >= 40 AND click_rate < 60 then 'okish'
           WHEN click_rate >= 60 then 'Splendid'
           ELSE NULL END as category
   FROM click_rate_table

In [ ]:
# Let's try to find change the amount of searches and clicks over each day.

WITH base_table AS (
    SELECT date,
    count(id),
    count(clickId),
    round((count(clickId)*1.0 / (count(id))) * 100, 2) as click_rate
    FROM (SELECT id,
                 datetime::date as date
          FROM searches) as s
                              
    LEFT JOIN (SELECT searchId,
                      clickId
              FROM clicks) c on c.searchId = s.id
              GROUP BY date), 
   
    click_rate_prev_day AS (
    SELECT *,
           LAG(click_rate, 1, 0) OVER (ORDER BY date) as click_rate_previous_day
    FROM base_table)


    SELECT *,
           (click_rate / click_rate_previous_day) - 1 as pct_change
    FROM click_rate_prev_day

    # ==> Note that I could have also done it this way (It's a little bit shorter):

WITH base_table AS (
    SELECT date,
    count(id) as num_searches,
    count(clickId) as num_clicks,
    round((count(clickId)*1.0 / (count(id))) * 100, 2) as click_rate
    FROM (SELECT id,
                 datetime::date as date
          FROM searches) as s
                              
    LEFT JOIN (SELECT searchId,
                      clickId
              FROM clicks) c on c.searchId = s.id
              GROUP BY date)
   
    SELECT *,
           (click_rate / LAG(click_rate, 1, 0) OVER (ORDER BY date)) - 1 as click_rate_previous_day
    FROM base_table)